In [5]:
#This script takes in the settings.json to act as input parameters for the rest of the scripts within this project.
#For variables which are not defined in the JSON, default parameters are used as defined within this script

import json
from datetime import datetime
from pytz import timezone
from skyfield.api import load
ts = load.timescale()
import re

class Settings:
    def __init__(self):
        #default parameter dictionary
        d = {"epoch": 0,
             "bstar": 0,
             "ndot": 0,
             "nddot": 0,
             "ecco": 0,
             "argpo": 0,
             "inclo": 0,
             "mo": 0,
             "no_kozai": 0.00437526951,
             "nodeo": 0,
             "timezone": 'US/Eastern',
             "start": "2000-01-01 00:00:00",
             "end": "2000-01-01 00:00:1",
             "lat": 38.8282,
             "lon": -77.3053,
             "elev": 140,
             "tdelta": 10,
             "chunks": 100,
             "tle1": "NA",
             "tle2": "NA",
             "t_eff": 0.5,
             "ccd_eff": 0.8,
             "t_diam": 0.8128,
             "beta": 0,
             "n": 0,
             "lat_loc": "38.8308",
             "lon_loc": "-77.3075",
             "humidity": "0.5"}

        #this sets the object attributes to the default values
        for k, v in d.items():
            setattr(self, k, v) 
        #replaces the objects default parametrs with any given value
        with open('settings.json') as f:
            variables = json.load(f)
        for key, value in variables.items():
            if value is not None and value != "":
                setattr(self, key, value)
        #uses regex to setup syntax used by future scripts for time keeping
        stimes = re.split("[-\s:]", self.start)
        itimes = [int(i) for i in stimes]
        time = datetime(itimes[0], itimes[1], itimes[2], itimes[3], itimes[4], itimes[5])
        time = ts.from_datetime(timezone(self.timezone).localize(time))
        setattr(self, "start", time)
        stimes = re.split("[-\s:]", self.end)
        itimes = [int(i) for i in stimes]
        time = datetime(itimes[0], itimes[1], itimes[2], itimes[3], itimes[4], itimes[5])
        time = ts.from_datetime(timezone(self.timezone).localize(time))
        setattr(self, "end", time)

parameters = Settings()


<>:53: SyntaxWarning: invalid escape sequence '\s'
<>:58: SyntaxWarning: invalid escape sequence '\s'
<>:53: SyntaxWarning: invalid escape sequence '\s'
<>:58: SyntaxWarning: invalid escape sequence '\s'
/tmp/ipykernel_24381/18383288.py:53: SyntaxWarning: invalid escape sequence '\s'
  stimes = re.split("[-\s:]", self.start)
/tmp/ipykernel_24381/18383288.py:58: SyntaxWarning: invalid escape sequence '\s'
  stimes = re.split("[-\s:]", self.end)


In [6]:
#NOTE: This entire script along with the flux, image sim, and settings code need to be put into jupiter notebooks and made runnable. Documentation must be completed

import time
import numpy as np
from sgp4.api import Satrec, WGS72
from skyfield.api import load, EarthSatellite, wgs84
from skyfield.positionlib import Barycentric
import pandas as pd
from pytz import timezone
from settings import parameters
import math

#This functions acts as an add on to calculate the solid angle overlap of the earth and the sun. This way we can calcualte how much of the sun is eclipsed by the earth and predict umbra/penubra
#However, since the script calculates single array elements instead of a whole array at once, it is very inefficient
#Allowing numpy to run these calculations would be far more efficient, and may drastically decrase runtime especially for the orbit propigator packeged elsewhere (which is a branch of this code)
def eclipse(satpos, earthpos, sunpos):

    #Nere we fetch planet positions at each time step and draw vectlrs to each
    satpos = Barycentric(satpos + earthpos).position.au
    satsun = sunpos - satpos
    satearth = earthpos - satpos
    satsundist = np.linalg.norm(satsun)
    satearthdist = np.linalg.norm(satearth)

    #calculating solid angle. I have no idea where those values for tan came from, will need to return to comment
    asun = math.atan(0.00465047/satsundist)
    aearth = math.atan(4.26354e-5/satearthdist)
    theta = math.acos(np.vdot(satsun, satearth)/(satearthdist * satsundist))    
    sunsolid = math.pi * asun ** 2
    earthsolid = math.pi * aearth ** 2

    #edge cases for whether or not to actually calculate the solid angle to save on runtime
    if (asun > theta + aearth):
        return str(round(earthsolid/sunsolid * 100)) + "%"
    elif (theta > asun + aearth or math.isclose(asun + aearth, theta)):
        return "0%"
    elif (aearth > asun + theta or math.isclose(asun + theta, aearth)):
        return "100%"
    #This is my own derivation of the problem, however a simple explanation of the problem can be found here https://www.youtube.com/watch?v=mMYCyeGVKUo
    #Will need to return when documenting to write up a mathmatical derivation
    #Fairly costly, however we luckly only run this code for a few minutes as the satelite is in penumbra
    else:
        a = (math.cos(aearth) - (math.cos(asun)*math.cos(theta)))/math.sin(theta)
        b = (math.sin(asun)**2 - a**2)**0.5
        
        p1 = np.array([0, 0, 1])
        p2 = np.array([math.sin(theta), 0, math.cos(theta)])
        p3 = np.array([a, -b, math.cos(asun)])
        p4 = np.array([a, b, math.cos(asun)])
                
        nb = np.cross(p1, p4 - p1)/np.linalg.norm(np.cross(p1, p4 - p1))
        nc = np.cross(p1, p3 - p1)/np.linalg.norm(np.cross(p1, p3 - p1))
        phi1 = math.acos(np.vdot(nb, nc))
        nb = np.cross(p2, p4 - p2)/np.linalg.norm(np.cross(p2, p4 - p2))
        nc = np.cross(p2, p3 - p2)/np.linalg.norm(np.cross(p2, p3 - p2))
        phi2 = math.acos(np.vdot(nb, nc))
        nb = np.cross(p4, p1 - p4)/np.linalg.norm(np.cross(p4, p1 - p4))
        nc = np.cross(p4, p3 - p4)/np.linalg.norm(np.cross(p4, p3 - p4))
        psi1 = math.acos(np.vdot(nb, nc))
        nb = np.cross(p4, p2 - p4)/np.linalg.norm(np.cross(p4, p2 - p4))
        nc = np.cross(p4, p3 - p4)/np.linalg.norm(np.cross(p4, p3 - p4))
        psi2 = math.acos(np.vdot(nb, nc))
                
        if (a < 0):
            digon = (2*math.pi-phi1)*(1-math.cos(asun)) + phi1 +2*psi1 - math.pi + phi2*(1-math.cos(aearth)) - (phi2+2*psi2-math.pi)
        else:
            digon = 2*math.pi - 2*(psi1 + psi2) - phi1 * math.cos(asun) - phi2 * math.cos(aearth)
        
        return str(round(digon/sunsolid * 100)) + "%"
    #dead code, should never fail unless inputs are fucked up. Only present for if
    return "fail"















#begining of script, we calculate runtime
start_time = time.time()
print("Simulating Satellite Orbit...")

ts = load.timescale()
#Calculating number of timesteps to calculate in the miliseconds
tscale = int((parameters.end - parameters.start) * 24 * 60 * 60 * 1000 + 1)

#loading the ephemeri of the sun and earth is intesive so its out here instead of the eclipse script
eph = load('de421.bsp')
sun, earth = eph['sun'], eph['earth']


# Initialize satellite using SGP4, for more information see https://pypi.org/project/sgp4/#providing-your-own-elements
sat = Satrec()
sat.sgp4init(
    WGS72,           # gravity model
    'i',             # 'a' = old AFSPC mode, 'i' = improved mode
    1,               # satnum: Satellite number
    parameters.epoch,               # epoch: days since 1949 December 31 00:00 UT
    parameters.bstar,           # bstar: drag coefficient (/earth radii)
    parameters.ndot,           # ndot: ballistic coefficient (radians/minute^2)
    parameters.nddot,             # nddot: second derivative of mean motion (radians/minute^3)
    parameters.ecco,             # ecco: eccentricity
    parameters.argpo,               # argpo: argument of perigee (radians)
    parameters.inclo,               # inclo: inclination (radians)
    parameters.mo,               # mo: mean anomaly (radians)
    parameters.no_kozai,  # no_kozai: mean motion (radians/minute) GEO
#    0.04908738521, #LEO
#    0.00872664625, #meo
    parameters.nodeo                # nodeo: right ascension of ascending node (radians)
)
#If TLE is present it will be used instead
if (parameters.tle1 != "NA" or parameters.tle2 != "NA"):
    sat = Satrec.twoline2rv(parameters.tle1, parameters.tle2)
# Convert Satrec object to EarthSatellite object
sat = EarthSatellite.from_satrec(sat, ts)

# Define the location of Observatory
obs = wgs84.latlon(parameters.lat, parameters.lon, parameters.elev)
# Vector between sat and obs
difference = sat - obs










#This chunking serves as a way for large amounts of time steps to be generated in single runs of the for loop.
#For some reason sgp4 cannot actually compute more than something like 10000 time intervals at once which is the point of the chuncking
#This chunking system has a fairly big bug where if the chuncks do not perfectly line up with the time intervals requested, it will fail
#to output the remaining time since it won't load the final chunk
#EXAMPLE: 
#input - chunk size = 3, start = 1, end = 8
#output - 1, 2, 3, 4, 5, 6
#         |______| |_____|  ...
#          chunk 1  chunk 2  no chunk 3
#As you can see times 7 and 8 were never output since chunk 3 couldn't fit.
chunk_size = parameters.tdelta * parameters.chunks
num_chunks = int(tscale/chunk_size)
satcords = np.zeros((num_chunks, 3, int(chunk_size / parameters.tdelta)), object)
satlatlon = np.zeros((num_chunks, 2, int(chunk_size / parameters.tdelta)), object)
obscords = np.zeros((num_chunks, 3, int(chunk_size / parameters.tdelta)), object)
eclipsepec = np.zeros((int(num_chunks*parameters.chunks), 1), object)
timelist = np.zeros((num_chunks, int(chunk_size / parameters.tdelta)), object)
tempdf = np.zeros((num_chunks, 5, int(chunk_size / parameters.tdelta)), object)
temptime = np.zeros((num_chunks, int(chunk_size / parameters.tdelta)), object)

for i in range(num_chunks):
    #Here we set an array of skyfield time objects to calculate positions at
    #the given chunked time arrays. We also convert the time into the given timezone after
   t = ts.utc(parameters.start.utc.year, \
              parameters.start.utc.month, \
              parameters.start.utc.day, \
              parameters.start.utc.hour, \
              parameters.start.utc.minute, \
              parameters.start.utc.second + np.arange(i*chunk_size, (i+1) * chunk_size, parameters.tdelta) * 0.001)
   timelist[i] = t
   temptime[i] = t.astimezone(timezone(parameters.timezone))
   
   #Calculating sattelite and observatory position at given time
   satcord = sat.at(t)
   obscoord = obs.at(t)
   satcords[i] = satcord.position.km
   obscords[i] = obscoord.position.km
   lat, lon = wgs84.latlon_of(satcord)
   satlatlon[i] = [lat.degrees, lon.degrees]
   
   #here is where we feed back into the eclipse function, calculating the posititions in array form.
   #This for loop is where I'm fairly sure the runtime is
   #getting caught during extensive runs. Running through each individual time
   #instead of letting numpy handle the entire array calculation is the issue.
   satpos = satcord.position.au
   sunpos = sun.at(t).position.au
   earthpos = earth.at(t).position.au
   for j in range(len(satpos[1])):
       eclipsepec[j + i*len(satpos[1])] = eclipse(satpos[:,j], earthpos[:,j], sunpos[:,j])

    #Calculating actual important info thats output for traking sattelite.
   topocentric = difference.at(t)
   ra, dec, distance = topocentric.radec()
   alt, az, trash = topocentric.altaz()
   tempdf[i] = np.array([ra._degrees, dec.degrees, az.degrees, alt.degrees, distance.km])

    #Just a useful check for how long its taking
   print('\r' + str(int(i/num_chunks * 10000)/100) + "%", end='', flush=True)


temptime = temptime.flatten()
print('\r' + "100.00%\n", end='', flush=True)  

#Output to data files for the input of future scripts and data analysis
df1 = pd.DataFrame({'Time (EST)': temptime, 
                   'RA (Deg)': tempdf[:, 0, :].flatten(), 'Dec (Deg)': tempdf[:, 1, :].flatten(),
                   'Az (Deg)': tempdf[:, 2, :].flatten(), 'Alt (Deg)': tempdf[:, 3, :].flatten(),
                   'Distance (Km)': tempdf[:, 4, :].flatten()})
df1["Eclipse %"] = eclipsepec.flatten().tolist()
df1.to_csv('satcoord.csv', index=False)

df = pd.DataFrame({'Lat': satlatlon[:, 0, :].flatten(), 'Lon': satlatlon[:, 1, :].flatten()})
df.to_csv('satlatlon.csv', index=False)
df = pd.DataFrame({'X': satcords[:, 0, :].flatten(), 'Y': satcords[:, 1, :].flatten(), 'Z': satcords[:, 2, :].flatten()})
df.to_csv('satcoordxyz.csv', index=False)

#End output to let know run is finished
end_time = time.time()
print("Simulation run complete and data stored...")
print('Execution time = %.6f seconds' % (end_time-start_time))


Simulating Satellite Orbit...
100.00%
Simulation run complete and data stored...
Execution time = 7.040191 seconds


In [9]:
import numpy as np
from scipy.integrate import quad
from settings import parameters
import sys
import matplotlib.pyplot as plt

"""
CODE FOR COMPUTING THE EXPECTED COUNTS AT A DETECTOR FROM THE LANDOLT SATELLITE WITH NO ATMOSPHERIC ABSORPTION
"""

data = np.genfromtxt('satcoord.csv',delimiter=',',skip_header=1) # inputs data file from orbit simulations
datalatlon = np.genfromtxt('satlatlon.csv',delimiter=',',skip_header=1) # inputs latitude and longitude information of the satellite
dataxyz = np.genfromtxt('satcoordxyz.csv',delimiter=',',skip_header=1) # inputs position data of the satellite

### VARIABLES ###

tdelta = parameters.tdelta/1000 # increment of time in the file loaded in
lat_obs = parameters.lat*(np.pi/180) # latitude of the center of the beam path
lon_obs = parameters.lon*(np.pi/180) # longitude of the center of the beam path
lat_loc = float(parameters.lat_loc)*(np.pi/180) # latitude of observer
lon_loc = float(parameters.lon_loc)*(np.pi/180) # longitude of observer
t_efficiency = float(parameters.t_eff) # telescope efficiency
ccd_efficiency = float(parameters.ccd_eff) # CCD efficiency
diam_t = float(parameters.t_diam) # diameter of telescope
a_t = np.pi*(diam_t/2)**2 # area that the telescope is able to take in light
lmbda_n = int(parameters.n) # determines which laser is being looked at (0 - 488nm, 1 - 785nm, 2 - 976nm, 3 - 1550nm)
humidity = float(parameters.humidity)
#ccd_sat = float(parameters.ccd_sat)
fob = 1 # frequency of blinking (in seconds)
aod = [0.055, 0.08, 0.06, 0.045, 0.045, 0.045, 0.035, 0.035]
# aod varies w/ humidity, the code factors that in here
if humidity >= 0.6:
    aod[lmbda_n] = aod[lmbda_n] + 0.05
if humidity >= 0.8:
    aod[lmbda_n] = aod[lmbda_n] + 0.05

z = data[:,5]*1e3 # extracting distance of observer to satellite in units of meters
alt = data[:,4]*(np.pi/180) # altitude of satellite in sky at the center of the beam path
alt0 = data[0,4]*(np.pi/180) # altitude of satellite in sky at any given location
alpha = np.pi/2 - alt # angle a line perpendicular to the center of the beam path makes with a tangent line located at the center of the beam path
t = np.linspace(0,len(z)-1,num=len(z))*tdelta # creates array of times incrementing with t=tdelta
airmass = (1/np.cos(alpha)) - 0.0018167*((1/np.cos(alpha))-1) - 0.002875*((1/np.cos(alpha))-1)**2 - 0.0008083*((1/np.cos(alpha))-1)**3 # airmass at a given altitude in the sky

orient_x = 6371000*np.sin((np.pi/2) - lat_obs)*np.cos(lon_obs) # location of center of beam path in the x direction using the volumetric mean radius of earth
orient_y = 6371000*np.sin((np.pi/2) - lat_obs)*np.sin(lon_obs) # similarly for the y direction
orient_z = 6371000*np.cos((np.pi/2) - lat_obs) # similarly for the z direction
orient_xloc = 6371000*np.sin((np.pi/2) - lat_loc)*np.cos(lon_loc) # location of the observer in the x direction using the volumetric mean radius of earth
orient_yloc = 6371000*np.sin((np.pi/2) - lat_loc)*np.sin(lon_loc) # similarly for the y direction
orient_zloc = 6371000*np.cos((np.pi/2) - lat_loc) # similarly for the z direction
orient_xloc = orient_xloc - orient_x # to get vector from center of the beam path to observer
orient_yloc = orient_yloc - orient_y # similarly
orient_zloc = orient_zloc - orient_z # similarly
sat_x = dataxyz[:,0] # x position of satellite in GCRS coordinates
sat_y = dataxyz[:,1] # y position of satellite in GCRS coordinates
sat_z = dataxyz[:,2] # z position of satellite in GCRS coordinates
sat_lat = datalatlon[:,0] # latitude of satellite projected to earth
sat_lon = datalatlon[:,1] # longitude of satellite projected to earth
r_sat = np.sqrt(sat_x**2 + sat_y**2 + sat_z**2)*1e3 # distance from satellite to the center of the earth in meters
orient_xsat = 6371000*np.sin((np.pi/2) - sat_lat)*np.cos(sat_lon) # location of the satellite in the x direction in lat/lon coordinates
orient_ysat = 6371000*np.sin((np.pi/2) - sat_lat)*np.sin(sat_lon) # similarly for the y direction
orient_zsat = 6371000*np.cos((np.pi/2) - sat_lat) # similarly for the z direction
orient_xsat = orient_xsat - orient_x
orient_ysat = orient_ysat - orient_y
orient_zsat = orient_zsat - orient_z
d0 = np.sqrt(orient_xloc**2 + orient_yloc**2 + orient_zloc**2) # distance of observer from center of beam path
beta = np.arccos(((orient_xloc*orient_xsat) + (orient_yloc*orient_ysat) + (orient_zloc*orient_zsat))/(d0*np.sqrt(orient_xsat**2 + orient_ysat**2 + orient_zsat**2))) # angle between a line made between the center of the beam path and observer and the satellite-to-center of beam path vector projected on to earth's surface
if d0 < diam_t/2:
    d0 = diam_t/2 # fixes error where starting at zero creates invalid variables, sets distance from center of the beam path to the radius of the telescope at the very minimum

w_z = np.zeros(len(z))
FWHM = np.zeros(len(z))
error_p = np.zeros(len(z))
z_new = np.zeros(len(t))
w_z = np.zeros(len(t))
flux_z = np.zeros(len(t))
tflux = np.zeros(len(t))
counts = np.zeros(len(t))
I_final = np.zeros(len(t))
counts_final = np.zeros(len(t))
mag_final = np.zeros(len(t))
num_flux = np.zeros(len(t))
num_counts = np.zeros(len(t))
num_I_final = np.zeros(len(t))
num_counts_final = np.zeros(len(t))
t_reqd = np.zeros(len(t))
num_t_reqd = np.zeros(len(t))
curve_theta = np.zeros(len(t))

MFD = [2.5e-6, 3.5e-6, 4e-6, 5e-6, 5.9e-6, 6.2e-6, 9.2e-6, 10.4e-6] # mode field diameter of optical fiber
w_0 = MFD[lmbda_n]/2 # waist radius of the gaussian beam
lmbda = [355e-9, 488e-9, 655e-9, 785e-9, 976e-9, 1064e-9, 1310e-9, 1550e-9] # wavelength of all eight lasers
P_0 = [0.003, 0.04, 0.05, 0.0636, 0.45, 0.3, 0.5, 0.1] # power of all four lasers
#P_0 = [0.1, 0.1, 0.1, 0.1, 0.1, 0.1, 0.1, 0.1]
zp = [417.5*1e-7*10000*a_t*1e10*lmbda[0]*1e-11, 632*1e-7*10000*a_t*1e10*lmbda[1]*1e-11, 217.7*1e-7*10000*a_t*1e10*lmbda[2]*1e-11, 112.6*1e-7*10000*a_t*1e10*lmbda[3]*1e-11, 31.47*1e-7*10000*a_t*1e10*lmbda[4]*1e-11, 31.47*1e-7*10000*a_t*1e10*lmbda[5]*1e-11, 31.47*1e-7*10000*a_t*1e10*lmbda[6]*1e-11, 11.38*1e-7*10000*a_t*1e10*lmbda[7]*1e-11] # zero points of each laser

### CALCULATIONS ###

I_0 = (2*P_0[lmbda_n])/(np.pi*w_0**2) # incident intensity of the laser
z_r = (np.pi/lmbda[lmbda_n])*w_0**2 # raleigh range

# calculates flux as a gaussian distribution for height above center of beam path given
w_z0 = w_0*np.sqrt(1+(z[0]/z_r)**2) # beam radius at distance z
if d0 > w_z0:
    print('Error: Observer Outside Beam Path')
    sys.exit()
FWHM = np.sqrt(2*np.log(2))*w_z0 # full width at half maximum of the beam profile for a given distance from the waist
x = np.arange(d0 - diam_t/2, d0 + diam_t/2, 0.001) # the distance on one direction perpendicular to the laser vector
theta = np.arctan(d0/z) # angle made between the normal of earth's surface and a beam of light landing a given distance away from the normal

print('Calculating Gaussian distribution of flux...')
for j in range(len(t)):
    curve_theta[j] = np.arctan(d0/6371000)
    z_new[j] = (z[j]+(6371000-6371000*np.cos(curve_theta[j])))/np.cos(theta[j]) # amount of distance a given light ray travels factoring in the curvature of the earth
    if alt0 <= alt[j]: # identifies if observer is closer or further from the satellite using its relative altitude in the sky
        z_new[j] = z_new[j] - d0*np.tan(alpha[j])*np.sin(beta[j])
    else:
        z_new[j] = z_new[j] + d0*np.tan(alpha[j])*np.sin(beta[j])
    w_z[j] = w_0*np.sqrt(1+(z_new[j]/z_r)**2) # beam radius observed on earth's surface accounting for the curvature of earth
    flux_z[j] = I_0*((w_0/w_z[j])**2)*np.e**((-2*d0**2)/w_z[j]**2) # flux along one 2D slice of the 3D gaussian beam profile for different distances from the satellite in the center of the beam path
print('Done!')

dis_t = x # distance of telescope from center of beam
error_p[1:] = z_new[1:]*0.00000484813681109536*t[1:] # error in pointing in km

print('Calculating flux recieved at telescope...') # finds the total flux over a given detector area
for i in range(len(t)):
    def flux_fn(r):
        return r*I_0*((w_0/w_z[i])**2)*np.e**((-2*r**2)/w_z[i]**2)
    tflux_temp = quad(flux_fn, -(diam_t/2) + d0, (diam_t/2) + d0) # flux taken in by a given telescope
    coeftemp = np.pi*(((diam_t/2) + d0)**2 - (-(diam_t/2) + d0)**2) / a_t # calculates fraction of distribution needed to be swept over to get an area a_t
    tflux[i] = tflux_temp[0]*(2*np.pi/coeftemp) # integrating over an angle that gives us an arclength of the diameter of the telescope
    counts[i] = (tflux[i]*lmbda[lmbda_n])/(6.62607015e-34*299792458) # total counts taken in
print('Done!')
"""
print('Calculating flux received at telescope numerically... (this will take a while)') # finds the total flux over a given detector area numerically
for i in range(len(t)):
    num_flux_temp = 0
    r_sum = 0
    for j in range(len(dis_t) - 1):
        def flux_fn(r): # defines the function for the distribution of light -> can be replaced with whatever
            return I_0*((w_0/w_z[i])**2)*np.e**((-2*r**2)/w_z[i]**2)    
        r_sum = flux_fn((dis_t[j+1] + dis_t[j])/2)*(dis_t[j+1] - dis_t[j]) # cycles through the area under the function of the distribution of light over the detector area
        num_flux_temp = num_flux_temp + r_sum # adds all areas of the curve to each other
    coeftemp = np.pi*(((diam_t/2) + (dis_t[0] + dis_t[len(dis_t)-1])/2)**2 - (-(diam_t/2) + (dis_t[0] + dis_t[len(dis_t)-1])/2)**2) / a_t # calculates fraction of distribution needed to be swept over to get an area a_t
    num_flux[i] = d0*num_flux_temp*(2*np.pi/coeftemp) # multiplies the 2d slice by the angle calculated above, then by the distance from the center of the beam path to successfully integrate it
    num_counts[i] = (num_flux[i]*lmbda[lmbda_n])/(6.62607015e-34*299792458) # converting flux to photoelectric counts
print('Done!')
"""
# Functions for calculating the scattering coefficient

def r_coef(cs, d, N):
    """
    Using the Rayleigh scattering cross section cs, the 
    scattering coefficient is found given the distance d from the satellite to the detector,
    and the amount of molecules N per cubic meter.
    """
    r_coef = np.e**(-cs*N*d)
    return r_coef

### CALCULATING INTENSITY WITH ATMOSPHERIC ABSORPTION ###

N = 1e44 / (((4/3)*np.pi*(6371000 + z_new)**3) - (4/3)*np.pi*(6371000)**3)# average number of molecules that measured light passes per square meter

# rayleigh scattering cross sections for 488 nm for the five most abundant gases in the atmosphere
if lmbda_n == 0:
    cs_n2 = 23.82e-31
    cs_o2 = 20.03e-31
    cs_ar = 23e-31
    cs_co2 = 70.70e-31
    cs_ne = 1.01e-31

# rayleigh scattering cross sections for 785 nm for the five most abundant gases in the atmosphere
elif lmbda_n == 1:
    cs_n2 = 7.26e-31
    cs_o2 = 6.50e-31
    cs_ar = 7.24e-31
    cs_co2 = 23e-31
    cs_ne = 0.33e-31

# rayleigh scattering cross sections for 976nm for the five most abundant gases in the atmosphere
# these values are extrapolated assuming a direct 1/lambda^4 relationship
elif lmbda_n == 2:
    cs_n2 = 2.24e-31
    cs_o2 = 2.06e-31
    cs_ar = 2.08e-31
    cs_co2 = 7.28e-31
    cs_ne = 0.103e-31

# rayleigh scattering cross sections for 1550nm for the five most abundant gases in the atmosphere
# these values are extrapolated assuming a direct 1/lambda^4 relationship
elif lmbda_n == 3:
    cs_n2 = 2.65e-31
    cs_o2 = 2.2e-31
    cs_ar = 2.38e-31
    cs_co2 = 6.22e-31
    cs_ne = 0.128e-31
    
elif lmbda_n == 4:
    cs_n2 = 1.11e-31
    cs_o2 = 0.92e-31
    cs_ar = 0.97e-31
    cs_co2 = 2.6e-31
    cs_ne = 0.128e-31
    
elif lmbda_n == 5:
    cs_n2 = 0.79e-31
    cs_o2 = 0.65e-31
    cs_ar = 0.68e-31
    cs_co2 = 1.84e-31
    cs_ne = 3.79e-33
    
elif lmbda_n == 6:
    cs_n2 = 0.34e-31
    cs_o2 = 0.28e-31
    cs_ar = 0.30e-31
    cs_co2 = 0.8e-31
    cs_ne = 1.65e-33
    
else:
    cs_n2 = 7.13e-33
    cs_o2 = 6.39e-33
    cs_ar = 7.11e-33
    cs_co2 = 2.26e-32
    cs_ne = 3.24e-34

r_coef1 = 1 - r_coef(cs_n2, z_new, N*0.78084) # scattering coefficient from rayleigh scattering for n2
r_coef2 = 1 - r_coef(cs_o2, z_new, N*0.20946) # scattering coefficient from rayleigh scattering for o2
r_coef3 = 1 - r_coef(cs_ar, z_new, N*0.00934) # scattering coefficient from rayleigh scattering for argon
r_coef4 = 1 - r_coef(cs_co2, z_new, N*0.000397) # scattering coefficient from rayleigh scattering for co2
r_coef5 = 1 - r_coef(cs_ne, z_new, N*1.818e-5) # scattering coefficient from rayleigh scattering for neon
m_coef = 1 - np.ones(len(t))*np.e**(-aod[lmbda_n]) # transmission coefficient from mie scattering

for i in range(len(t)):
    I_final[i] = (tflux[i] - tflux[i]*(m_coef[i]+r_coef1[i]+r_coef2[i]+r_coef3[i]+r_coef4[i]+r_coef5[i])*airmass[i])*t_efficiency # calculates flux observed at telescope
    counts_final[i] = ((I_final[i]*lmbda[lmbda_n])/(6.62607015e-34*299792458))*ccd_efficiency # total counts taken in
    mag_final[i] = -2.5*np.log10(I_final[i]/zp[lmbda_n]) # relative magnitude calculated from vega zero points
    num_I_final[i] = (num_flux[i] - num_flux[i]*(m_coef[i]+r_coef1[i]+r_coef2[i]+r_coef3[i]+r_coef4[i]+r_coef5[i])*airmass[i])*t_efficiency # calculates numerical flux observed at telescope
    num_counts_final[i] = ((num_I_final[i]*lmbda[lmbda_n])/(6.62607015e-34*299792458))*ccd_efficiency # conversion from numerically calculated flux to photoelectric counts
    t_reqd[i] = 4.4e5 / counts_final[i] # amount of seconds needed to observe 4.4e5 counts
    num_t_reqd[i] = 4.4e5 / num_counts_final[i] # same as above but for the numerical counts

### FORMATTING FILE FOR EXPORT ###

for i in range(int(fob/1e-3)):
    I_final[i::2*int(fob/1e-3)] = 0
    counts_final[i::2*int(fob/1e-3)] = 0
    mag_final[i::2*int(fob/1e-3)] = 0
    num_I_final[i::2*int(fob/1e-3)] = 0
    num_counts_final[i::2*int(fob/1e-3)] = 0
    
I_final = I_final[0:len(t)]
counts_final = counts_final[0:len(t)]
mag_final = mag_final[0:len(t)]

# code for outputting the array of flux values
heading = np.array('Radiant Flux (W)',dtype='str')
heading2 = np.array('Counts per Second')
heading3 = np.array('Airmass')
heading4 = np.array('Magnitude')
heading5 = np.array('Recommended Exposure Time (s)')
data = np.genfromtxt('satcoord.csv',dtype='str',delimiter=',') # inputs data file
data = data[:,:7]
I_final = np.asarray(I_final,dtype='str')
counts_final = np.asarray(counts_final,dtype='str')
airmass = np.asarray(airmass,dtype='str')
mag_final = np.asarray(mag_final,dtype='str')
t_reqd = np.asarray(t_reqd,dtype='str')
I_final = np.insert(I_final[:],0,heading)
counts_final = np.insert(counts_final[:],0,heading2)
airmass = np.insert(airmass[:],0,heading3)
mag_final = np.insert(mag_final[:],0,heading4)
t_reqd = np.insert(t_reqd[:],0,heading5)
output = np.column_stack((data,I_final))
output = np.column_stack((output,counts_final))
output = np.column_stack((output,airmass))
output = np.column_stack((output,mag_final))
output = np.column_stack((output,t_reqd))
np.savetxt('satcoord.csv', output, fmt='%s', delimiter=',')



Calculating Gaussian distribution of flux...
Done!
Calculating flux recieved at telescope...
Done!


/tmp/ipykernel_24381/2912773482.py:240: RuntimeWarning: divide by zero encountered in scalar divide
  num_t_reqd[i] = 4.4e5 / num_counts_final[i] # same as above but for the numerical counts
